### For evaluation, we need to build the model again(not training)

if model is not saved as keras then model should be rebuiltas original and checkpoints should be loaded

In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" # Enable legacy Keras behavior ignoring keras-3 as model and training checkpoints are saved in legacy keras format

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

2026-01-15 13:16:52.065002: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-15 13:16:55.807528: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
IMG_SIZE = 384
NUM_CLASSES = 101
BATCH_SIZE = 32
BUFFER_SIZE = 1000 

In [4]:
import tensorflow_datasets as tfds
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal"),
                                         tf.keras.layers.RandomRotation(0.2),
                                         tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2)],                                      
                                         # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetV2B0
                                        name ="data_augmentation")


# Preprocessing image
def preprocess_fn(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)    
    return image, label


def build_training_dataset(dataset, BUFFER_SIZE, BATCH_SIZE):    
    # shuffle images before batching up
    dataset_processed = (
                            dataset
                            .shuffle(BUFFER_SIZE)
                            .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
                            .map(lambda x, y: (data_augmentation(x, training=True), y))
                            .batch(BATCH_SIZE)
                            .prefetch(1)
                        )
    return dataset_processed

def build_eval_dataset(dataset, BATCH_SIZE):
    return (
        dataset
        .map(preprocess_fn, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(1)
    )   


I0000 00:00:1768479417.868436   10350 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5518 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:64:00.0, compute capability: 8.9


In [5]:
def build_food101_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=101, dropout_rate=0.5):

    # Setup base model, freezing the base model layers and considering pretrained weights on imagenet
    base_model = tf.keras.applications.EfficientNetV2S(include_top=False, 
                                                       weights="imagenet", 
                                                       input_shape=input_shape
                                                       )  
    
    inputs = tf.keras.layers.Input(shape=input_shape, name="input_layer")
 
    a1 = base_model(inputs, training=False)   # training = True, only last 20 layers are trainable,  
                                          # training = False, BatchNorms and Dropouts are freezed during training  very important

    # Hybrid pooling
    gap = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(a1)
    gmp = tf.keras.layers.GlobalMaxPooling2D(name="max_pooling_layer")(a1)
    a2 = tf.keras.layers.Concatenate(name="max_avg_pooling")([gap, gmp])

    # Dropout
    a3 = tf.keras.layers.Dropout(dropout_rate)(a2)  # add dropout layer and a bit stronger on small data
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", dtype="float32", name="output_layer")(a3)

    return tf.keras.Model(inputs, outputs), base_model

model, base_model = build_food101_model(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=101, dropout_rate=0.5)


In [6]:
model.load_weights("/mnt/d/04_Food101-EfficientNetV2S-model/models/checkpoints/ckpt_model_100p.ckpt")

In [7]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name="top1"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5"),
    ],
)


In [8]:
# Export the model in SavedModel format for TensorFlow Serving(keras-3 compatible) not native keras-3 .keras format
model.save(
    "/mnt/d/04_Food101-EfficientNetV2S-model/models/export/food101_savedmodel",
    save_format="tf"
)

INFO:tensorflow:Assets written to: /mnt/d/04_Food101-EfficientNetV2S-model/models/export/food101_savedmodel/assets


INFO:tensorflow:Assets written to: /mnt/d/04_Food101-EfficientNetV2S-model/models/export/food101_savedmodel/assets
